In [ ]:
import os
import glob
import numpy as np
import tifffile as tiff
import cv2
import matplotlib.pyplot as plt
from skimage.measure import label
from edf import EDF

#   Funciones de Ground Truth
def binarize_and_count(gt_path):
    """Carga GT, binariza todo >0 como célula y cuenta objetos."""
    img = tiff.imread(gt_path)

    if img.ndim == 3:
        img = img[0]  # GT a veces viene como stack

    if img.dtype == np.uint16:
        img = np.clip(img, 0, 255).astype(np.uint8)

    # Binarización simple
    _, binary = cv2.threshold(img, 1, 255, cv2.THRESH_BINARY)

    # Conteo de objetos conectados
    num_labels = label(binary).max()
    return img, binary, num_labels

#   Funciones para YOLO
def normalize_16bit_to_8bit(img):
    p1, p99 = np.percentile(img, (1, 99))
    img = np.clip(img, p1, p99)
    img = (img - img.min()) / (img.max() - img.min())
    return (img * 255).astype(np.uint8)

def process_for_yolo(image_path):
    img_stack = tiff.imread(image_path)
    original = img_stack[0] if img_stack.ndim == 3 else img_stack
    img_norm = normalize_16bit_to_8bit(original) if original.dtype == np.uint16 else original.copy()
    img_yolo = cv2.cvtColor(img_norm, cv2.COLOR_GRAY2BGR) if img_norm.ndim == 2 else img_norm.copy()
    return original, img_yolo

def draw_boxes(image, boxes):
    img_drawn = image.copy()
    for box in boxes:
        x1, y1, x2, y2 = map(int, box[:4])
        conf = box[4]
        cv2.rectangle(img_drawn, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(img_drawn, f'{conf:.2f}', (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
    return img_drawn

#   Pipeline completo: EDF + GT + visualización + guardado
def process_and_save(image_path, gt_path, model, output_dir):
    # Procesamiento GT
    gt_original, gt_binary, gt_count = binarize_and_count(gt_path)

    # Procesamiento YOLO
    orig, processed = process_for_yolo(image_path)
    boxes = model(processed)
    img_yolo = draw_boxes(processed, boxes)

    # Combinar GT binarizado y YOLO en un mismo canvas
    canvas = np.zeros((orig.shape[0], orig.shape[1]*2, 3), dtype=np.uint8)

    # A la izquierda: GT binarizado
    gt_rgb = cv2.cvtColor(gt_binary, cv2.COLOR_GRAY2BGR)
    cv2.putText(gt_rgb, f'GT Count: {gt_count}', (10,20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    canvas[:, :orig.shape[1], :] = gt_rgb

    # A la derecha: YOLO detecciones
    cv2.putText(img_yolo, f'YOLO Det: {len(boxes)}', (10,20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    canvas[:, orig.shape[1]:, :] = img_yolo

    # Guardar imagen final como JPG
    base_name = os.path.splitext(os.path.basename(image_path))[0] + ".jpg"
    save_path = os.path.join(output_dir, base_name)
    cv2.imwrite(save_path, canvas)

if __name__ == "__main__":
    # Directorios
    data_dir = "../Fluo-N2DL-HeLa (1)/Fluo-N2DL-HeLa/02"
    gt_dir = "../Fluo-N2DL-HeLa (1)/Fluo-N2DL-HeLa/02_GT/TRA"
    output_dir = "./output/visual/consensus-0.2"
    os.makedirs(output_dir, exist_ok=True)

    # Inicializar EDF
    model = EDF(
        'cfg/yolov3.cfg',
        'weights/edf',
        ensemble_option='consensus',
        conf_threshold=0.2
    )

    # Listar imágenes y GT
    image_files = sorted(glob.glob(os.path.join(data_dir, "*.tif")))
    gt_files = sorted(glob.glob(os.path.join(gt_dir, "*.tif")))

    # Procesar todas las imágenes
    for img_path, gt_path in zip(image_files, gt_files):
        print(f'Procesando {os.path.basename(img_path)}')
        process_and_save(img_path, gt_path, model, output_dir)

    print(f'Todas las imágenes guardadas en {output_dir}')
